In [580]:
#===============================================================================
# class Map
#===============================================================================

class Random:
  """
  class Random: portable pseudo random generator for random integers modulo N
  >>> rg = Random(seed=88)             # create random generator
  >>> rg.rand()                        # get integer random number
  871
  >>> rg.matrix(m=2,n=3,N=6)           # get random int matrix modulo N
  array([[4., 0., 2.],
         [5., 1., 3.]])
  """
  def __init__(self,seed):
    self.p = 1 + 2*3*5*7*11*13         # small prime number
    self.P = -1 + 2*3*5*7*11*13*17*19  # large prime number
    self.seed = seed+1

  def rand(self):
    self.seed = ((self.seed+1) * self.P) % self.p;
    return self.seed

#===============================================================================
# class Map
#===============================================================================

class Map:
    def __init__(self,cluster=None):
        self.cluster = cluster
        self.shape = cluster.shape if cluster is not None else (4,10,2,5)

    def kappa(self,i,j=None):
        """
        self.kappa():  convert matrix indices to linear index or vice versa
        >>> Map().kappa(i:=1,j:=3)   # k = i+j*m
        13
        >>> Map().kappa(k:=13)       # i = k%m, j = k//m
        (1, 3)
        """

        m,n,d,s = self.shape
        if j is None:
            k = i
            return (k%m,k//m)
        else:
            return i + j*m

    def permanence(self,p):    # encode permanence
        """
        self.permanence(p): convert permanence to symbolic string
        >>> o = Map()
        >>> o.permanence(0.52)
        'B'
        >>> o.permanence([-1,0,0.01,0.49,0.5,0.99,1,2])
        '<0yaAY1>'
        """
        def upper(x):
            return chr(int(65+(x-0.5)*100//2))
        def lower(x):
            return chr(int(65+32+(0.5-x)*100//2))

        if isinstance(p,list):
            s = ''
            for k in range(len(p)):
                s += self.permanence(p[k])
            return s

        if p < 0:
            return '<'
        elif p == 0:
            return '0'
        elif p == 1:
            return '1'
        elif p > 1:
            return '>'
        elif p < 0.5:
            return lower(p)
        elif p >= 0.5:
            return upper(p)
        else:
            return '?'

    def symbol(self,x):
        """
        self.symbol(x): convert index to symbol or vice versa
        >>> o = Map()
        >>> o.symbol(11)
        'B'
        >>> o.symbol([0,1,10,11,35,36,37,61,62])
        '01ABZabz062'
        """
        def symb(x):
            if x < 10:
                return chr(48+x)
            if x < 36:
                return chr(55+x)
            elif x < 62:
                return chr(61+x)
            else:
                return '%03g' % x

        if isinstance(x,int):
            return symb(x)
        elif isinstance(x,list):
            s = ''
            for k in range(len(x)):
                s += self.symbol(x[k])
            return s

    def bar(self,n,label='',k=-1):          # bar string of length n
            if n >= 5:
                if k >= 0:
                    str = '%03g' % k
                    if len(label) > 0:
                        str += '/' + label
                else:
                    str = '---'
                while len(str) < n:
                    str += '-'
                    if len(str) < n: str = '-' + str
                return str
            if n >= 3:
                label = '-' + label
            elif n >= 5:
                label = '-' + label
            str = label
            for k in range(n-len(label)): str += '-'
            return str

    def head(self,i,n,s,width=0):
        line = '+'
        s = max(s,width)
        for j in range(n):
            if i < 0:
                sym = ''
                line += self.bar(s,'') + '+'
            else:
                k = self.kappa(i,j)
                sym = self.symbol(k)
                line += self.bar(s,sym,k) + '+'
        return line

    def table(self,kind,I,m,n,width=0,label=''):    # print table
        """
        self.table('i',...) # for indices
        self.table('p',...) # for permanences
        self.table('w',...) # for synaptic weights
        """
        def title(n,x):
            return '-%03g-' % x

        def row(kind,I,i,j,d,s,width):
            str = ''
            for nu in range(s):
               if kind == 'i':   # index
                   str += self.symbol(I[nu])
               elif kind == 'p': # permanence
                   str += self.permanence(I[nu])
               elif kind == 'w': # permanence
                   str += '1' if I[nu] > 0.5 else '0'
               else:
                   str += '?'
            while len(str) < width:
                str = str + ' '
                if len(str) < width: str = ' ' + str
            return str

        cells = self.cluster
        d = len(I[0][0])
        s = len(I[0][0][0])

        tab = ''
        for k in range(len(label)): 
            tab += ' '
        
        str = ''
        for i in range(m):
            head = self.head(i,n,s,width)
            trailer = label if i == 0 else tab
            print(trailer+head)
            for mu in range(d):
                line = tab + '|'
                for j in range(n):
                    line += row(kind,I[i][j][mu],i,j,mu,s,width) + '|'
                print(line)
        print(tab+self.head(-1,n,s,width))
   
    def Pmap(self):
        m,n,d,s = cells.shape
        self.table('p',self.cluster.P,m,n,width=max(s,7),label='P: ')

    def Kmap(self):
        m,n,d,s = cells.shape
        self.table('i',self.cluster.K,m,n,width=max(s,7),label='K: ')

    def Gmap(self):
        m,n,d,s = cells.shape
        self.table('i',self.cluster.G,m,n,width=max(s,7),label='G: ')

    def Wmap(self):
        m,n,d,s = cells.shape
        self.table('w',self.cluster.P,m,n,width=max(s,7),label='W: ')


In [600]:
"""
class Cluster(): parameter definition for neurotron clusters
clu = Cluster(m,n,d,s)
"""

from numpy import array

#=========================================================================
# class Cluster
#=========================================================================

class Cluster:
    def __init__(self,m=4,n=10,d=2,s=5,f=10,g=3):
        self.shape = (m,n,d,s)
        self.P = self.tensor(m,n,d,s)     # prediction permanences
        self.K = self.tensor(m,n,d,s)     # prediction indices
        self.F = self.tensor(m,n,1,f)     # excitation weights
        self.G = self.tensor(m,n,1,g)     # collaboration weights
        
        self.U = array(self.zeros(m,n))
        self.Q = array(self.zeros(m,n))
        self.D = array(self.zeros(m,n))
        self.B = array(self.zeros(m,n))
        self.X = array(self.zeros(m,n))
        self.Y = array(self.zeros(m,n))
        self.S = array(self.zeros(m,n))
        self.L = array(self.zeros(m,n))

    def zeros(self,d,s):
        return [[0 for j in range(s)] for i in range(d)]
    
    def tensor(self,m,n,d,s):
        return [[self.zeros(d,s) for j in range(n)] for i in range(m)]
            
    def kappa(self,i,j=None):
        """
        self.kappa():  convert matrix indices to linear index or vice versa
        >>> o = Cluster(m=4,n=10)
        >>> k = o.kappa(i:=1,j:=3)  # k = i+j*m 
        13
        >>> ij = o.kappa(k:=13)    # i = k%m, j = k//m
        (1,3)
        """

        m,n,d,s = self.shape
        if j is None:
            k = i
            return (k%m,k//m)
        else:
            return i + j*m

    def setP(self,i,j,P):
        assert isinstance(P,list) and isinstance(P[0],list) 
        self.P[i][j] = P.copy()

    def setK(self,i,j,K):
        assert isinstance(K,list) and isinstance(K[0],list) 
        self.K[i][j] = K.copy()

    def setG(self,i,j,G):
        assert isinstance(G,list) and isinstance(G[0],list) 
        self.G[i][j] = G.copy()
    
    def initG(self):
        m,n,d,s = self.shape
        for i in range(m):
            for j in range(n):
                g = []
                for nu in range(m):
                    k = self.kappa(nu,j)
                    if nu != i: g.append(k)
                self.setG(i,j,[g])

    def initK(self):
        rg = Random(seed=0)             # create random generator
        m,n,d,s = self.shape
        N = m*n
        for i in range(m):
            for j in range(n):
                K = self.zeros(d,s)
                for mu in range(d):
                    for nu in range(s):
                        K[mu][nu] = rg.rand() % N
                self.setK(i,j,K)

    def initP(self):
        rg = Random(seed=0)             # create random generator
        m,n,d,s = self.shape
        Q = 20                          # quantizing constant
        for i in range(m):
            for j in range(n):
                P = self.zeros(d,s)
                for mu in range(d):
                    for nu in range(s):
                        P[mu][nu] = (1 + rg.rand() % Q) / Q
                self.setP(i,j,P)

    def init(self):
        self.initP()
        self.initK()
        self.initG()
        return self

    def map(self):
        Map(cells).Pmap()
        Map(cells).Kmap()
        Map(cells).Gmap()
        

In [604]:
cells = Cluster(4,10,2,5).init()
cells.map()


P: +-000/0-+-004/4-+-008/8-+-012/C-+-016/G-+-020/K-+-024/O-+-028/S-+-032/W-+-036/a-+
   | prhmC | PwJCW | JhkC1 | whhme | hkWMM | ECwkH | wJeJc | EAhcW | uuUh1 | heP1r |
   | RUuUW | EHrEM | Uk1kU | EpREc | wJmck | RJrh1 | RRCMW | eRRPm | UUmcr | eeeCP |
   +-001/1-+-005/5-+-009/9-+-013/D-+-017/H-+-021/L-+-025/P-+-029/T-+-033/X-+-037/b-+
   | AMwC1 | CUwAE | UpEAA | mCRUu | CWEHr | C1Uk1 | meEpR | MMwJm | kHRJr | JcRRC |
   | kwkAE | eHUmk | Juprh | UWPwJ | EMJhk | kUwhh | EchkW | ckECw | h1wJe | MWEAh |
   +-002/2-+-006/6-+-010/A-+-014/E-+-018/I-+-022/M-+-026/Q-+-030/U-+-034/Y-+-038/c-+
   | cWeRR | h1UUm | 1reee | C1kwk | AEeHU | AAJup | UuUWP | HrEMJ | k1kUw | pREch |
   | PmuuU | crheP | CPAMw | AECUw | mkUpE | rhmCR | wJCWE | hkC1U | hhmeE | kWMMw |
   +-003/3-+-007/7-+-011/B-+-015/F-+-019/J-+-023/N-+-027/R-+-031/V-+-035/Z-+-039/d-+
   | JmckE | Jrh1w | RCMWE | RRPmu | Umcrh | eeCPA | wkAEC | HUmkU | uprhm | WPwJC |
   | CwkHR | JeJcR | AhcWe | uUh1U | eP1re | MwC1k | UwAEe | pEAA

In [606]:
cells = Cluster(1,3,1,3).init()
cells.map()


P: +-000/0-+-001/1-+-002/2-+
   |  prh  |  mCR  |  UuU  |
   +-------+-------+-------+
K: +-000/0-+-001/1-+-002/2-+
   |  100  |  110  |  111  |
   +-------+-------+-------+
G: +-000/0-+-001/1-+-002/2-+
   |       |       |       |
   +-------+-------+-------+
